# Experiment 04: Amazon Planet

This experiment uses the data from the Kaggle competition [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/leaderboard). Here we use a pretrained ResNet50 model to generate the features from the dataset.

For details of virtual machine we used and the versions of LightGBM and XGBoost, please refer to [experiment 1](01_airline.ipynb).

In [44]:
import sys
from collections import defaultdict
import numpy as np
import pkg_resources
from libs.loaders import load_planet_kaggle
from libs.planet_kaggle import threshold_prediction
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


In [2]:
%env MOUNT_POINT=/datadrive

env: MOUNT_POINT=/datadrive


The images are loaded and featurised using a pretrained ResNet50 model available from Keras

In [3]:
X_train, y_train, X_test, y_test = load_planet_kaggle()

Featurising training images: 100%|██████████| 1094/1094.0 [36:54<00:00,  1.91s/it]
Featurising validation images: 100%|██████████| 172/172.0 [05:48<00:00,  1.57s/it]


In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(35000, 2048)
(35000, 17)
(5479, 2048)
(5479, 17)


## XGBoost vs LightGBM benchmark

We will compare both libraries on speed and preformance.

In [45]:
number_processors = get_number_processors()
print("Number of processors: ", number_processors)

Number of processors:  24


We will use a one-v-rest. So each classifier will be responsible for determining whether the assigned tag applies to the image

In [46]:
def train_and_validate_xgboost(params, train_features, train_labels, validation_features, num_boost_round):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    time_results = defaultdict(list)
    for class_i in tqdm(range(n_classes)):
        dtrain = xgb.DMatrix(data=train_features, label=train_labels[:, class_i])
        dtest = xgb.DMatrix(data=validation_features)
        with Timer() as t:
            model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
        time_results['train_time'].append(t.interval)
        
        with Timer() as t:
            y_val_pred[:, class_i] = model.predict(dtest)
        time_results['test_time'].append(t.interval)
        
    return y_val_pred, time_results

In [47]:
def train_and_validate_lightgbm(model, train_features, train_labels, validation_features):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    time_results = defaultdict(list)
    for class_i in tqdm(range(n_classes)):
        with Timer() as t:
            model.fit(train_features, train_labels[:, class_i])
        time_results['train_time'].append(t.interval)
        
        with Timer() as t:
            y_val_pred[:, class_i] = model.predict(validation_features)
        time_results['test_time'].append(t.interval)
        
    return y_val_pred, time_results

In [48]:
metrics_dict = {
    'Accuracy': accuracy_score,
    'Precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='samples'),
    'Recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='samples'),
    'F1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='samples'),
}

def classification_metrics(metrics, y_true, y_pred):
    return {metric_name:metric(y_true, y_pred) for metric_name, metric in metrics.items()}

In [49]:
results_dict = dict()
num_rounds = 50

In [62]:
xgb_params = {'max_depth':6, 
              'objective':'binary:logistic', 
              'min_child_weight':1, 
              'eta':0.1, 
              'colsample_bytree':0.80,
              'scale_pos_weight':2, 
              'gamma':0.1, 
              'reg_lamda':1, 
              'subsample':1,
              'nthread':number_processors
             }

In [63]:
y_pred, timing_results = train_and_validate_xgboost(xgb_params, X_train, y_train, X_test, num_boost_round=num_rounds)

100%|██████████| 17/17 [05:06<00:00, 18.03s/it]


In [64]:
results_dict['xgb']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

In [53]:
xgb_hist_params = {'max_depth':0, 
                  'objective':'binary:logistic', 
                  'min_child_weight':1, 
                  'eta':0.1, 
                  'colsample_bytree':0.80,
                  'scale_pos_weight':2, 
                  'gamma':0.1, 
                  'reg_lamda':1, 
                  'subsample':1,
                  'nthread':number_processors,
                  'tree_method':'hist', 
                  'max_leaves':2**6, 
                  'grow_policy':'lossguide',
                  'max_bins': 63
                 }

In [55]:
y_pred, timing_results = train_and_validate_xgboost(xgb_hist_params, X_train, y_train, X_test, num_boost_round=num_rounds)


100%|██████████| 17/17 [37:33<00:00, 123.12s/it]


In [56]:
results_dict['xgb_hist']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

In [58]:
lgbm_model = LGBMClassifier(num_leaves=2**6, 
                           learning_rate=0.1, 
                           scale_pos_weight=2,
                           n_estimators=number_processors,
                           min_split_gain=0.1,
                           min_child_weight=1,
                           reg_lambda=1,
                           subsample=1,
                           nthread=number_processors,
                           max_bin=63)

In [59]:
y_pred, timing_results = train_and_validate_lightgbm(lgbm_model, X_train, y_train, X_test)

100%|██████████| 17/17 [01:18<00:00,  3.58s/it]


In [60]:
results_dict['lgbm']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

/anaconda/envs/strata/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda/envs/strata/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [65]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "Accuracy": 0.5831356086877167,
            "F1": 0.8916251743227468,
            "Precision": 0.8966667101226329,
            "Recall": 0.9062528789577522
        },
        "test_time": 0.2287097891094163,
        "train_time": 78.43849972297903
    },
    "xgb": {
        "performance": {
            "Accuracy": 0.3423982478554481,
            "F1": 0.8057198664184697,
            "Precision": 0.7188249796481225,
            "Recall": 0.9782772316811376
        },
        "test_time": 0.17806852096691728,
        "train_time": 291.9369727639714
    },
    "xgb_hist": {
        "performance": {
            "Accuracy": 0.3694104763643001,
            "F1": 0.8199552360766837,
            "Precision": 0.7411964006871837,
            "Recall": 0.9724867242023657
        },
        "test_time": 0.16979735810309649,
        "train_time": 2238.962064931169
    }
}


This dataset shows an interesting behavior. It is the only notebook where XGBoost hist behaves worse than XGBoost. The reason could be because the number of features is high, 2048, and that could be causing a memory overhead. LightGBM and the standard version of XGBoost can manage this high number of features, so there is no overhead. You can try to use a higher complexity to improve the performance. For example, setting `max_depth=8` in XGBoost, `max_leaves=2**8` in XGBoost hist and `num_leaves=2**6` in LightGBM. This will cause an overhead in XGBoost hist.